In [15]:
import pandas as pd

In [1]:
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [3]:
import mlflow

In [4]:
TRACKING_SERVER_HOST = "ec2-3-12-83-113.us-east-2.compute.amazonaws.com"

In [5]:
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [6]:
mlflow.set_experiment('nyc-taxi-experiment')

<Experiment: artifact_location='s3://ml-mlflow-artifact/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    if filename.endswith(".csv"):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 70)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [16]:
df_train = read_dataframe("../../dataset/green_tripdata_2022-01.parquet")
df_val = read_dataframe("../../dataset/green_tripdata_2022-02.parquet")

In [17]:
df_train.shape, df_val.shape

((59837, 21), (66488, 21))

In [18]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

In [19]:
val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [20]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

8.846638935176887

In [20]:
with open('../../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [22]:
# Track the experiments with MLflow
from statistics import mean


with mlflow.start_run():
    mlflow.set_tag("developer", "alok")

    mlflow.log_param("train-data-path", "../../dataset/green_tripdata_2022-01.parquet")
    mlflow.log_param("train-data-path", "../../dataset/green_tripdata_2022-01.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    ls = Lasso(alpha)
    ls.fit(X_train, y_train)

    y_pred = ls.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="../../models/lin_reg.bin", artifact_path="models_pickle")

In [12]:
import xgboost as xgb

In [13]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [21]:
# create the xgboost matrix
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [23]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
    
    return {"loss": rmse, "status": STATUS_OK}

In [27]:
search_space = {
    "max_depth": scope.int(hp.quniform('max_depth', 4, 100, 1)),
    "learning_rate": hp.loguniform('learning_rate', -3, 0),
    "reg_alpha": hp.loguniform('reg_alpha', -5, -1),
    "reg_lambda": hp.loguniform('reg_lambda', -6, -1),
    "min_child_weight": hp.loguniform('min_child_weight', -1, 3),
    "objective": "reg:linear",
    "seed": 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[13:30:59] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.79524                          
[1]	validation-rmse:15.64152                          
[2]	validation-rmse:14.60149                          
[3]	validation-rmse:13.66579                          
[4]	validation-rmse:12.82839                          
[5]	validation-rmse:12.07871                          
[6]	validation-rmse:11.40530                          
[7]	validation-rmse:10.81037                          
[8]	validation-rmse:10.27807                          
[9]	validation-rmse:9.80703                           
[10]	validation-rmse:9.38918                          
[11]	validation-rmse:9.02009                          
[12]	validation-rmse:8.69576                          
[13]	validation-rmse:8.40637                          
[14]	validation-rmse:8.15756                          
[15]	validation-rmse:7.93521                          
[

In [22]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

In [25]:
params = {
    'learning_rate':0.15615748360138743,
    'max_depth':4,
    'min_child_weight':2.350889641981104,
    'objective': 'reg:linear',
    'reg_alpha':0.10619618494552516,
    'reg_lambda':0.028152089722374665,
    'seed':42
}

In [27]:
with mlflow.start_run():
    mlflow.xgboost.autolog()
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )

[17:51:02] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:15.81877
[1]	validation-rmse:13.96549
[2]	validation-rmse:12.45058
[3]	validation-rmse:11.22586
[4]	validation-rmse:10.23899
[5]	validation-rmse:9.45159
[6]	validation-rmse:8.83068
[7]	validation-rmse:8.34710
[8]	validation-rmse:7.96530
[9]	validation-rmse:7.66837
[10]	validation-rmse:7.43825
[11]	validation-rmse:7.26202
[12]	validation-rmse:7.12089
[13]	validation-rmse:7.01155
[14]	validation-rmse:6.92811
[15]	validation-rmse:6.85827
[16]	validation-rmse:6.80946
[17]	validation-rmse:6.76493
[18]	validation-rmse:6.72401
[19]	validation-rmse:6.69424
[20]	validation-rmse:6.67036
[21]	validation-rmse:6.65638
[22]	validation-rmse:6.63670
[23]	validation-rmse:6.62256
[24]	validation-rmse:6.61245
[25]	validation-rmse:6.60173
[26]	validation-rmse:6.59062
[27]	validation-rmse:6.58235
[28]	validation-rmse:6.57489
[29]	validation-rmse:6.57016
[30]	validation-

2022/06/28 17:51:35 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2022/06/28 17:51:38 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/anaconda3/envs/ml-ops/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


In [29]:
for model_class in (RandomForestRegressor, GradientBoostingRegressor, LinearSVR):
    with mlflow.start_run():
        mlflow.sklearn.autolog()
        mlflow.log_param("train-data-path", "../../dataset/green_tripdata_2022-01.parquet")
        mlflow.log_param("valid-data-path", "../../dataset/green_tripdata_2022-02.parquet")
        mlflow.log_artifact("../../models/", artifact_path="sklearn-models/")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)